In [1]:
import numpy as np
import pandas as pd
import requests
from io import StringIO
from bs4 import BeautifulSoup as BS
from time import sleep
from tqdm import tqdm
import re

In [2]:
fda_df = pd.read_csv('Data/fdalabel-query-327575.csv')

In [3]:
fda_df.head(10)

,Labeling Type,Dosage Form(s),Route(s) of Administration,Marketing Category,Application Number(s),Trade Name,Generic/Proper Name(s),Most Recent SPL Date (YYYY/MM/DD),Established Pharmacologic Class(es),Initial U.S. Approval,...,Marketing Date(s) (YYYY/MM/DD),NDC(s),Active Ingredient UNII(s),Active Ingredient(s),Active Moiety Name(s),Active Moiety UNII(s),FDALabel Link,DailyMed Link,DailyMed PDF Link,SET ID
0,HUMAN PRESCRIPTION DRUG LABEL,SOLUTION,ORAL,NDA,212690,XYWAV,"(CALCIUM, MAGNESIUM, POTASSIUM, AND SODIUM OXY...",20210823,Central Nervous System Depressant,2002.0,...,2020/11/02-,68727-150,7G33012534; 8W24SYD6ZI; G983HLV265; S8NKF3H3KT,Calcium Oxybate; Magnesium Oxybate; Potassium ...,4-HYDROXYBUTANOIC ACID,30IW36W5B2,http://nctr-crs.fda.gov/fdalabel/services/spl/...,http://dailymed.nlm.nih.gov/dailymed/lookup.cf...,http://dailymed.nlm.nih.gov/dailymed/downloadp...,1e0ae43a-037f-42af-8e23-a0e51d75abe8
1,HUMAN PRESCRIPTION DRUG LABEL,"INJECTION, SOLUTION",INFILTRATION; PERINEURAL,NDA,009435,(CHLOROPROCAINE HCI,(CHLOROPROCAINE HCI,20200222,Ester Local Anesthetic,NaN,...,2019/10/13-,51662-1407,LT7Z1YW11H,CHLOROPROCAINE HYDROCHLORIDE,CHLOROPROCAINE; EDETIC ACID,5YVB0POT2H; 9G34HU7RV0,http://nctr-crs.fda.gov/fdalabel/services/spl/...,http://dailymed.nlm.nih.gov/dailymed/lookup.cf...,http://dailymed.nlm.nih.gov/dailymed/downloadp...,94d13f48-fbca-67b4-e053-2995a90a791b
2,HUMAN PRESCRIPTION DRUG LABEL,SOLUTION,AURICULAR (OTIC),NDA,208251,OTOVEL,(CIPROFLOXACIN AND FLUOCINOLONE ACETONIDE),20190912,Corticosteroid; Quinolone Antimicrobial,2016.0,...,2016/04/29-,24338-080,0CD5FD6S2M; 5E8K9I0O4U,CIPROFLOXACIN; FLUOCINOLONE ACETONIDE,CIPROFLOXACIN; FLUOCINOLONE ACETONIDE,0CD5FD6S2M; 5E8K9I0O4U,http://nctr-crs.fda.gov/fdalabel/services/spl/...,http://dailymed.nlm.nih.gov/dailymed/lookup.cf...,http://dailymed.nlm.nih.gov/dailymed/downloadp...,1a7e4806-3138-4b7a-b798-a99e163ee846
3,HUMAN PRESCRIPTION DRUG LABEL,KIT,INTRAVENOUS,BLA,125611,REBINYN,"(COAGULATION FACTOR IX (RECOMBINANT), GLYCOPEG...",20200630,Human Blood Coagulation Factor,2017.0,...,2017/12/01-,0169-7901; 0169-7902; 0169-7905,382L14738L,COAGULATION FACTOR IX RECOMBINANT HUMAN,COAGULATION FACTOR IX RECOMBINANT HUMAN,382L14738L,http://nctr-crs.fda.gov/fdalabel/services/spl/...,http://dailymed.nlm.nih.gov/dailymed/lookup.cf...,http://dailymed.nlm.nih.gov/dailymed/downloadp...,0ea37235-35fd-410d-b8c4-40ba15fe1294
4,HUMAN PRESCRIPTION DRUG LABEL,"INJECTION, POWDER, LYOPHILIZED, FOR SUSPENSION",INTRAVENOUS,NDA,209401,VYXEOS,(DAUNORUBICIN AND CYTARABINE) LIPOSOME,20210413,Anthracycline Topoisomerase Inhibitor; Nucleos...,2017.0,...,2017/08/03-,68727-745,04079A1RDZ; ZS7284E0ZP,CYTARABINE; DAUNORUBICIN,CYTARABINE; DAUNORUBICIN,04079A1RDZ; ZS7284E0ZP,http://nctr-crs.fda.gov/fdalabel/services/spl/...,http://dailymed.nlm.nih.gov/dailymed/lookup.cf...,http://dailymed.nlm.nih.gov/dailymed/downloadp...,7ea701ce-e7d3-4349-a9c2-642a501d45c8
5,HUMAN PRESCRIPTION DRUG LABEL,INJECTION,INTRAVENOUS,BLA,761107,GAMIFANT,(EMAPALUMAB-LZSG),20200630,Interferon gamma Blocker,2018.0,...,2019/05/17-; 2020/06/26-,66658-501; 66658-505; 66658-510,3S252O2Z4X,emapalumab,emapalumab,3S252O2Z4X,http://nctr-crs.fda.gov/fdalabel/services/spl/...,http://dailymed.nlm.nih.gov/dailymed/lookup.cf...,http://dailymed.nlm.nih.gov/dailymed/downloadp...,a865e0ef-8685-4f69-8838-648c4f3bab47
6,HUMAN PRESCRIPTION DRUG LABEL,"INJECTION, SOLUTION",SUBCUTANEOUS,BLA,208583,Xultophy 100/3.6,(INSULIN DEGLUDEC AND LIRAGLUTIDE),20191115,GLP-1 Receptor Agonist; Insulin Analog,2016.0,...,2016/11/21-,0169-2911,54Q18076QB; 839I73S42A,INSULIN DEGLUDEC; LIRAGLUTIDE,INSULIN DEGLUDEC; LIRAGLUTIDE,54Q18076QB; 839I73S42A,http://nctr-crs.fda.gov/fdalabel/services/spl/...,http://dailymed.nlm.nih.gov/dailymed/lookup.cf...,http://dailymed.nlm.nih.gov/dailymed/downloadp...,21335fe4-d395-4501-ac2a-2f20d7520da9
7,HUMAN PRESCRIPTION DRUG LABEL,"CAPSULE, EXTENDED RELEASE",ORAL,NDA,205831,APTENSIO XR,(METHYLPHENIDATE HYDROCHLORIDE),20210630,Central Nervou

In [4]:
fda_df['Trade Name'] = fda_df['Trade Name'].str.lower()

In [5]:
fda_df = fda_df.sort_values('Most Recent SPL Date (YYYY/MM/DD)', ascending = False)

In [6]:
fda_df.drop_duplicates(subset='Trade Name')

,Labeling Type,Dosage Form(s),Route(s) of Administration,Marketing Category,Application Number(s),Trade Name,Generic/Proper Name(s),Most Recent SPL Date (YYYY/MM/DD),Established Pharmacologic Class(es),Initial U.S. Approval,...,Marketing Date(s) (YYYY/MM/DD),NDC(s),Active Ingredient UNII(s),Active Ingredient(s),Active Moiety Name(s),Active Moiety UNII(s),FDALabel Link,DailyMed Link,DailyMed PDF Link,SET ID
10855,HUMAN PRESCRIPTION DRUG LABEL,TABLET,ORAL,Export only,NaN,cyproheptadine hydrochloride,CYPROHEPTADINE HYDROCHLORIDE,2010110126,NaN,NaN,...,2010/06/15-,62033-0346,NJ82J0F8QC,Cyproheptadine Hydrochloride,Cyproheptadine,2YHB6175DO,http://nctr-crs.fda.gov/fdalabel/services/spl/...,http://dailymed.nlm.nih.gov/dailymed/lookup.cf...,http://dailymed.nlm.nih.gov/dailymed/downloadp...,f90faaa3-f40f-4653-8fb8-144d61c6b1d4
17740,HUMAN PRESCRIPTION DRUG LABEL,"TABLET, FILM COATED",ORAL,ANDA,077525,gabapentin,GABAPENTIN,20220321,Anti-epileptic Agent,1993.0,...,2006/08/24-,71610-069,6CW7F3G59X,GABAPENTIN,GABAPENTIN,6CW7F3G59X,http://nctr-crs.fda.gov/fdalabel/services/spl/...,http://dailymed.nlm.nih.gov/dailymed/lookup.cf...,http://dailymed.nlm.nih.gov/dailymed/downloadp...,7790d906-af26-4e60-bc6f-e906ec03cc24
22866,HUMAN PRESCRIPTION DRUG LABEL,"TABLET, FILM COATED",ORAL,ANDA,078106,levetiracetam,LEVETIRACETAM,20220108,NaN,1999.0,...,2012/09/11-,43547-221; 43547-222; 43547-223; 43547-224,44YRR34555,LEVETIRACETAM,LEVETIRACETAM,44YRR34555,http://nctr-crs.fda.gov/fdalabel/services/spl/...,http://dailymed.nlm.nih.gov/dailymed/lookup.cf...,http://dailymed.nlm.nih.gov/dailymed/downloadp...,dd713e60-478c-43cd-b680-ff594a0b3864
5728,HUMAN PRESCRIPTION DRUG LABEL,"TABLET, FILM COATED, EXTENDED RELEASE",ORAL,ANDA,077415,bupropion hydrochloride,BUPROPION HYDROCHLORIDE,20220101,Aminoketone,1985.0,...,2009/05/01-,63187-713,ZG7E5POY8O,BUPROPION HYDROCHLORIDE,BUPROPION,01ZG3TPX31,http://nctr-crs.fda.gov/fdalabel/services/spl/...,http://dailymed.nlm.nih.gov/dailymed/lookup.cf...,http://dailymed.nlm.nih.gov/dailymed/downloadp...,e57451fd-aba5-47d6-9892-ffe2a6021b6f
17439,HUMAN PRESCRIPTION DRUG LABEL,"INJECTION, POWDER, LYOPHILIZED, FOR SOLUTION",INTRAVENOUS,ANDA,204025,fosaprepitant,FOSAPREPITANT,20211231,Substance P/Neurokinin-1 Receptor Antagonist,2008.0,...,2020/10/22-,16729-240,D35FM8T64X,FOSAPREPITANT DIMEGLUMINE,APREPITANT,1NF15YR6UY,http://nctr-crs.fda.gov/fdalabel/services/spl/...,http://dailymed.nlm.nih.gov/dailymed/lookup.cf...,http://dailymed.nlm.nih.gov/dailymed/downloadp...,4a3bac1c-1702-4963-b967-f1699f03cc6f
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33647,HUMAN PRESCRIPTION DRUG LABEL,CAPSULE,ORAL,NaN,NaN,urobiotic,"OXYTETRACYCLINE HYDROCHLORIDE, SULFAMETHIZOLE ...",20051214,NaN,NaN,...,NaN,0049-0920,25W8454H16; 2IUY41693Z; 4U7K4N52ZM,oxytetracycline hydrochloride; phenazopyridine...,oxytetracycline; phenazopyridine; sulfamethizole,25W8454H16; X20I9EN955,http://nctr-crs.fda.gov/fdalabel/services/spl/...,http://dailymed.nlm.nih.gov/dailymed/lookup.cf...,http://dailymed.nlm.nih.gov/dailymed/downloadp...,be27854a-a805-4300-9729-accd1b7f226f
35293,HUMAN PRESCRIPTION DRUG LABEL,TABLET,ORAL,NaN,NaN,renese,POLYTHIAZIDE,20051214,NaN,NaN,...,NaN,0069-3750; 0069-3760; 0069-3770,36780APV5N,polythiazide,polythiazide,36780APV5N,http://nctr-crs.fda.gov/fdalabel/services/spl/...,http://dailymed.nlm.nih.gov/dailymed/lookup.cf...,http://dailymed.nlm.nih.gov/dailymed/downloadp...,abd6ecf0-dc8e-41de-89f2-1e36ed9d6535
33645,HUMAN PRESCRIPTION DRUG LABEL,CAPSULE,ORAL,NaN,NaN,terramycin,OXYTETRACYCLINE HYDROCHLORIDE,20051207,NaN,NaN,...,NaN,0069-0730,4U7K4N52ZM,Oxytetracycline Hydrochloride,Oxytetracycline,X20I9EN955,http://nctr-crs.fda.gov/fdalabel/services/spl/...,http://dailymed.nlm.nih.gov/dailymed/lookup.cf...,http://dailymed.nlm.nih.gov/dailymed/downloadp...,6b4de8c9-8a32-4a07-9183-419d6177ea08
8197,HUMAN PRESCRIPTION DRUG LABEL,CAPSULE,ORAL,NaN,NaN,chlordiazepoxide hcl,CHLORDIAZEPOXIDE HYDROCHLORIDE,2005

In [7]:
fda_df['webscraping_name'] = fda_df['Trade Name'].str.extract('(\w+)')

In [8]:
pd.set_option('display.max_colwidth', None)
fda_df['FDALabel Link']

10855    http://nctr-crs.fda.gov/fdalabel/services/spl/set-ids/f90faaa3-f40f-4653-8fb8-144d61c6b1d4/spl-doc
17740    http://nctr-crs.fda.gov/fdalabel/services/spl/set-ids/7790d906-af26-4e60-bc6f-e906ec03cc24/spl-doc
22866    http://nctr-crs.fda.gov/fdalabel/services/spl/set-ids/dd713e60-478c-43cd-b680-ff594a0b3864/spl-doc
5728     http://nctr-crs.fda.gov/fdalabel/services/spl/set-ids/e57451fd-aba5-47d6-9892-ffe2a6021b6f/spl-doc
17439    http://nctr-crs.fda.gov/fdalabel/services/spl/set-ids/4a3bac1c-1702-4963-b967-f1699f03cc6f/spl-doc
                                                        ...                                                
26675    http://nctr-crs.fda.gov/fdalabel/services/spl/set-ids/40e35fa9-9cfe-04f1-e054-00144ff8d46c/spl-doc
40691    http://nctr-crs.fda.gov/fdalabel/services/spl/set-ids/1cd07b54-f66f-4040-9058-462324267196/spl-doc
22289    http://nctr-crs.fda.gov/fdalabel/services/spl/set-ids/40d145e6-13f7-41ac-e054-00144ff8d46c/spl-doc
36580    http://nctr-crs.fda

In [43]:
drugz = 'teflaro'

In [44]:
endpoint = f'https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/{drugz}-side-effects.html'

In [45]:
response = requests.get(endpoint)

In [46]:
response

<Response [200]>

In [47]:
soup = BS(response.text)

In [48]:
soup.find_all(['h3','i','li'])

[<i class="icon icon-cross"></i>,
 <li>
 <a href="https://web.archive.org/web/20160608042942/https://www.facebook.com/Drugscom" target="_blank"><i class="icon icon-facebook"></i></a>
 <a href="https://web.archive.org/web/20160608042942/https://twitter.com/drugscom" target="_blank"><i class="icon icon-twitter"></i></a>
 <a href="https://web.archive.org/web/20160608042942/https://plus.google.com/+drugscom?rel=publisher" target="_blank"><i class="icon icon-googleplus"></i></a>
 </li>,
 <i class="icon icon-facebook"></i>,
 <i class="icon icon-twitter"></i>,
 <i class="icon icon-googleplus"></i>,
 <li class="nav_item dropnav menu_account logged-out">
 <a class="nav_link" href="https://web.archive.org/web/20160608042942/https://www.drugs.com/account/login/" rel="nofollow"><i class="icon icon-user"></i>Sign In</a>
 <div class="nav_folder">
 <ul class="sub-menu">
 <li><a class="input-button btn-block" href="https://web.archive.org/web/20160608042942/https://www.drugs.com/account/login/" rel="n

In [15]:
#COME BACK TO THIS TO SEE IF ITS POSSIBLE. THE WAY TO DO IT WOULD BE GRAB FIRST 2 H3, THEN GRAB THE Is and put them into LIST,
#THEN GRAB LIs UNDER EACH I AND PUT THEM INTO A LIST, THEN ZIP LISTS 
# list1 = []
# for span in soup.find_all(['h3','i', 'li'], attrs={'class':None},string = True):
#      text = span.next_element

#      if text:
#          list1.append(span)

        
# x = list1.index('More common')
# y = list1.index('General')
# list1[x:y]
        
        

In [16]:
list1 = soup.find_all(['h3','i', 'li'], attrs={'class':None},string = True)

In [17]:
str_list1 = [str(i) for i in list1]

In [18]:
#index1 = str_list1.index('<h3>Side effects requiring immediate medical attention</h3>')

In [19]:
#index2 = str_list1.index([x for x in str_list1 if f'<i>Applies to {drugz}' in x][0])

In [20]:
#q = list1[index1:index2][0]

In [21]:
#q.name


In [71]:
h3 = ''
i = ''
li = ''
SE_df = pd.DataFrame()
results = []
for drugz in  list(fda_df['webscraping_name']):
    URL123 = f'https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/{drugz}-side-effects.html'
    res1 = requests.get(URL123)
    soup = BS(res1.text)
    list1 = soup.find_all(['h3','i', 'li'], attrs={'class':None},string = True)
    print(URL123)
    str_list1 = [str(i) for i in list1]
    if ('<h3>Side effects requiring immediate medical attention</h3>' in str_list1) & ('<h3>General</h3>' in str_list1):
        index_first = str_list1.index('<h3>Side effects requiring immediate medical attention</h3>')
        #index_last = str_list1.index([x for x in str_list1 if f'<i>Applies to {drugz}' in x][0])
        index_last = str_list1.index('<h3>General</h3>')
        for ele in list1[index_first:index_last]:
            if ele.name == 'h3':
                h3 = ele.text
            elif ele.name == 'i':
                i = ele.text
            else:
                li = ele.text
                results.append((drugz,h3,i,li))
    elif ('<h3>Side effects requiring immediate medical attention</h3>' in str_list1) :
        if ele.name == 'h3':
                h3 = ele.text
        elif ele.name == 'i':
                i = ele.text
        else:
                li = ele.text
                results.append((drugz,h3,i,li))
    else:
        results.append((drugz,'NA','NA','NA'))

https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/cyproheptadine-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/gabapentin-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/levetiracetam-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/bupropion-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/fosaprepitant-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/paclitaxel-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/teflaro-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/dimethyl-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/aricept-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/pravastatin-side-effects.html
https://web.archiv

https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/cephalexin-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/clobetasol-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/ciprofloxacin-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/promethazine-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/eszopiclone-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/pantoprazole-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/gabapentin-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/diazepam-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/gabapentin-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/spironolactone-side-effects.html
https://we

https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/metformin-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/prednisone-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/prednisone-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/tramadol-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/metronidazole-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/onfi-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/penicillin-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/diethylpropion-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/sertraline-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/metronidazole-side-effects.html
https://web.archive

https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/menopur-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/skelaxin-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/prevacid-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/pristiq-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/glipizide-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/phentermine-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/warfarin-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/phentermine-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/dicyclomine-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/phentermine-side-effects.html
https://web.archive.org/web/2

https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/lorazepam-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/kaletra-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/miglitol-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/amoxicillin-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/lexapro-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/gabapentin-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/diovan-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/carvedilol-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/carbatrol-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/penicillamine-side-effects.html
https://web.archive.org/web/20

...

https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/oxygen-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/oxygen-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/oxygen-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/nitrous-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/nitrous-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/oxygen-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/oxygen-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/nitrous-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/oxygen-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/helium-side-effects.html
https://web.archive.org/web/20211121022625/https://www.dr

https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/air-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/carbon-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/oxygen-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/air-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/oxygen-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/air-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/oxygen-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/oxygen-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/oxygen-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/oxygen-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/

https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/nifedipine-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/famotidine-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/acetazolamide-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/hydroxyzine-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/sildenafil-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/levothyroxine-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/candesartan-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/levothyroxine-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/phytonadione-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/prohance-side-effects.html
https://w

https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/omega-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/zepzelca-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/oxygen-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/hydroxyzine-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/gabapentin-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/neurontin-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/sildenafil-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/carbon-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/valacyclovir-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/valacyclovir-side-effects.html
https://web.archive.org/web/202

ConnectionError: HTTPSConnectionPool(host='web.archive.org', port=443): Max retries exceeded with url: /web/20211121022625/https://www.drugs.com/sfx/oxygen-side-effects.html (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000023E2A12DFA0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [72]:
SE_df = pd.DataFrame(results)

In [75]:
SE_df = SE_df.rename(columns={0:'webscraping_name', 2:'medical_attention',2:'commonality', 3:'Side_effect'})

In [77]:
SE_df = SE_df.rename(columns = {1:'medical_attention'})

In [81]:
SE_df

,webscraping_name,medical_attention,commonality,Side_effect
0,cyproheptadine,NA,NA,NA
1,gabapentin,Side effects requiring immediate medical attention,More common,Clumsiness or unsteadiness
2,gabapentin,Side effects requiring immediate medical attention,More common,"continuous, uncontrolled, back-and-forth, or rolling eye movements"
3,gabapentin,Side effects requiring immediate medical attention,More common in children,anxiety
4,gabapentin,Side effects requiring immediate medical attention,More common in children,concentration problems and change in school performance
...,...,...,...,...
155257,hydroxyzine,Side effects not requiring immediate medical attention,Incidence not known,Dry mouth
155258,hydroxyzine,Side effects not requiring immediate medical attention,Incidence not known,headache
155259,hydroxyzine,Side effects not requiring immediate medical attention,Incidence not known,"seeing, hearing, or feeling things that are not there"
155260,brompheniramine,NA,NA,NA


In [83]:
SE_df.to_csv('Data/SE_df.csv')

In [ ]:
side_efx = soup.find('p', attrs = {'class':False})

In [ ]:
side_efx = side_efx.text

In [ ]:
side_efx = side_efx.split('.')

In [ ]:
side_efx[0:2]
common_se = side_efx[0]
other_se = side_efx[1]

In [ ]:
common_se = common_se.split(':')
other_se = other_se.split(':')

In [ ]:
def Convert(lst):
    res_dct = {lst[i]: lst[i + 1] for i in range(0, len(lst), 2)}
    return res_dct

In [ ]:
common_se_dict = Convert(common_se)
other_se_dict = Convert(other_se)

In [ ]:
common_se_dict

In [ ]:
fda_df.head()

In [ ]:
# drug_name = {}
# sfx = {'webscraping_name':[],'Side Effects':[]}
# common_sfx = []
# other_sfx = []
# drug_side_efx = pd.DataFrame()
# for drug in list(fda_df['webscraping_name'][0:10]):
#     URL = f'https://www.drugs.com/sfx/{drug}-side-effects.html'
#     res = requests.get(URL)
#     soup_obj = BS(res.text)
#     drug_name = {'webscraping_name':f'{drug}'}
#     side_effect = soup_obj.find('p', attrs = {'class':False})
#     side_eff = side_effect.text
#     #side_efx = side_eff.split('.')
# #     common_sfx = side_efx[0]
# #     other_sfx = side_efx[1]
# #     common_sfx= common_sfx.split(':')
# #     other_sfx = other_sfx.split(':')
# #     common_sfx_dict = Convert(common_sfx)
# #     other_sfx_dict = Convert(other_sfx)
# #     common_sfx_dict.update(other_sfx_dict)
# #     common_sfx_dict.update(drug_name)
#     sfx['webscraping_name'].append(f'{drug}')
#     sfx['Side Effects'].append(side_eff)
     
    
    

In [ ]:
# sfx_df = pd.DataFrame(sfx)

In [ ]:
# sfx_df

In [ ]:
# sfx_df[sfx_df['Side Effects'].str.contains('Applies')]

In [ ]:
# sfx_df

In [ ]:
# URL = 'https://www.drugs.com/sfx/cyproheptadine-side-effects.html'
# res = requests.get(URL)
# soup_obj = BS(res.text)

In [ ]:
# soup_obj.find_all('p', text = True)

In [ ]:
# common_sfx_dict

In [ ]:
# adverse_rxn.head()

In [ ]:
# food_df = pd.read_csv('Data/food.csv')

In [ ]:
food_nutrient_df = pd.read_csv('Data/food_nutrient.csv')

In [ ]:
nutrient_df = pd.read_csv('Data/nutrient.csv').rename(columns = {'id':'nutrient_id'})

In [ ]:
food_and_nutrient = pd.merge(food_nutrient_df, nutrient_df, on = 'nutrient_id')

In [ ]:
food_and_nutrient.head()

In [ ]:
food = pd.merge(food_df,food_and_nutrient, on = 'fdc_id')

In [ ]:
food.head()

In [ ]:
food = food.drop(columns = ['food_category_id','min','max','median','loq', 'footnote', 'min_year_acquired'])

In [ ]:
food.head()

In [ ]:
#se_df = pd.read_table('Data/meddra_all_label_se.tsv')

In [ ]:
#drug_code = pd.read_table('Data/drug_atc.tsv')

In [ ]:
#drug_names = pd.read_table('Data/drug_names.tsv')

In [ ]:
#drug_names.head(10)

In [ ]:
# fda_df[fda_df['Trade Name']=='CARNITINE']

In [ ]:
# THISURL = 'https://www.drugs.com/alpha/condition/a'

In [ ]:
# res1 = requests.get(THISURL)

In [ ]:
# soup_1 = BS(res1.text)

In [ ]:
# conditions = soup_1.select('.ddc-list-column-2 a')

In [ ]:
# conditions_list = list(conditions)

In [ ]:
# re.search(r'(>\w+\s?\w+?<)', str(conditions_list[1]))

In [ ]:
# conditions_list

In [ ]:
# x = len(conditions_list)
# cond_list = []
# for i in range(0,x):
#     cond_list.append(re.search(r'(>\w+\W?\s?\w+?\s?\w+?\s?\W?\w+?\W?<)', str(conditions_list[i])).group(1))
#     condition_list.append(cond_list[i].replace('>','')
#     cond_list[i].replace('<','')

In [ ]:
# cond_list

In [ ]:
set